In [10]:
import tensorflow as tf

# Cargar imágenes desde directorio
dataset = tf.keras.utils.image_dataset_from_directory(
    './data',
    image_size=(128, 128),  # Tamaño al que se redimensionan las imágenes
    batch_size=32,
    label_mode='categorical'  # Cambiar a 'int' si necesitas etiquetas como enteros
)

# Dividir en entrenamiento y validación
train_ds = dataset.take(int(0.8 * len(dataset)))
val_ds = dataset.skip(int(0.8 * len(dataset)))

# Normalizar imágenes
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

Found 700 files belonging to 2 classes.


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Rescaling, GlobalAveragePooling2D

# Entrada de la imagen
input_image = Input(shape=(128, 128, 3))

# Primera capa convolucional (similar a las primeras capas de YOLO)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_image)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Segunda capa convolucional
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Tercera capa convolucional
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Cuarta capa convolucional (aumentando la profundidad de la red)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Capa de aplanado
x = Flatten()(x)

# Capa densa
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)

# Capa final para clasificación, usando softmax para clases múltiples
output = Dense(train_ds.element_spec[1].shape[1], activation='softmax')(x)

# Crear el modelo
model = Model(inputs=input_image, outputs=output)

# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
model.summary()

# Entrenamiento del modelo
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,  # Puedes ajustar el número de épocas
)

# Evaluar el modelo
loss, accuracy = model.evaluate(val_ds)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

# Guardar el modelo entrenado
model.save('modelo_yolov8_inspirado.h5')


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_16 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 64, 64, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 32, 32, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_18 (Conv2D)          (None, 32, 32, 128)       7385